In [1]:
import cv2
import numpy as np
import os
from random import shuffle
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
IMG_SIZE=96

# region of interest (ROI) coordinates
top, right, bottom, left = 100, 150, 400, 450

a=''
arr = ['ycbcr','edge detection using filters', 'HSV' , 'hsv with histogram back projection']
color_model=int(input('enter the image preocesing model to use 1)ycbcr 2)edge detection using filters 3)HSV 4)hsv with histogram back projection : '))

dir0 = arr[color_model-1]
print(dir0)
try:
    os.mkdir(dir0)
except:
    print('contain folder in same name')

# get the reference to the webcam
camera = cv2.VideoCapture(0)

enter the image preocesing model to use 1)ycbcr 2)edge detection using filters 3)HSV 4)hsv with histogram back projection :  1


ycbcr
contain folder in same name


In [3]:
def convolve(B, r):
    D = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(r,r))
    cv2.filter2D(B, -1, D, B)
    return B

In [4]:
def img_process(image_frame,color_model):        
    if color_model==1:
        gray = cv2.cvtColor(image_frame, cv2.COLOR_BGR2YCR_CB)
        #gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
        skin_ycrcb_min = np.array((0, 133, 77))
        skin_ycrcb_max = np.array((255, 173, 127))
        gray = cv2.inRange(gray, skin_ycrcb_min, skin_ycrcb_max);  # detecting the hand in the bounding box using skin detection 
        contours,hierarchy = cv2.findContours(gray.copy(),cv2.RETR_EXTERNAL, 2)
        #resize img
        gray = cv2.GaussianBlur(gray, (7, 7), 0)
        ret, gray = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        gray = cv2.resize(gray, (IMG_SIZE,IMG_SIZE))
        
    elif color_model==2:
        gray = cv2.cvtColor(image_frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),2)
        th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
        ret, gray = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        gray = cv2.resize(gray, (IMG_SIZE,IMG_SIZE))
    
    elif color_model==3:
        min_HSV = np.array([0, 58, 30], dtype = "uint8")
        max_HSV = np.array([33, 255, 255], dtype = "uint8")
        # Get pointer to video frames from primary device
        image = image_frame
        imageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        skinRegionHSV = cv2.inRange(imageHSV, min_HSV, max_HSV)
        skinHSV = cv2.bitwise_and(image, image, mask = skinRegionHSV)
        gray = np.hstack([image, skinHSV])
        gray = cv2.GaussianBlur(gray, (7, 7), 0)
        ret, gray = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        gray = cv2.resize(gray, (IMG_SIZE,IMG_SIZE))
        
    elif color_model==4:
        image_hsv = cv2.cvtColor(image_frame,cv2.COLOR_BGR2HSV)
        M = cv2.calcHist([image_hsv], channels=[0, 1], mask=None, histSize=[80, 256], ranges=[0, 180, 0, 256] )
        B = convolve(M, r=5)
        ret, gray = cv2.threshold(th3, 30, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return gray

In [ ]:
while(True):

    a=input('exit: e or enter the label name : ')

    if a=="e":
        break

    dir1=str(dir0)+'/'+str(a)
    print(dir1)

    try:
        os.mkdir(dir1)
    except:
        print('contain folder')

    i=0

    while(True):
        (t, frame) = camera.read()

        

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # get the ROI
        roi = frame[top:bottom, right:left]

        gray = img_process(roi,color_model)
    
        #write img file to directory
        cv2.imwrite("%s/%s/%d.jpg"%(dir0,a,i),gray)
        i+=1
        print(i)
        if i>500:
            break

        # draw the segmented hand
        cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)

        cv2.imshow("Video Feed 1", gray)

        cv2.imshow("Video Feed", frame)
        # observe the keypress by the user
        keypress = cv2.waitKey(1)

        # if the user pressed "Esc", then stop looping
        if keypress == 27:
            break

# free up memory
camera.release()
cv2.destroyAllWindows()

exit: e or enter the label name :  e


In [6]:
path=dir0
IMG_SIZE = 96
def create_train_data():
    training_data = []
    label=0
    for (dirpath,dirnames,filenames) in os.walk(path):
        for dirname in dirnames:
            print(dirname)
            for(direcpath,direcnames,files) in os.walk(path+"/"+dirname):
                for file in files:
                        actual_path=path+"/"+dirname+"/"+file
                        #print(files)
                        # label=label_img(dirname)
                        path1 =path+"/"+dirname+'/'+file
                        img=cv2.imread(path1,cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                        training_data.append([np.array(img),label])
            label=label+1
            print(label)
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    print(training_data)
    #return training_data

In [7]:
create_train_data()

A
1
B
2
C
3
D
4
E
5
F
6
G
7
H
8
I
9
J
10
K
11
L
12
M
13
N
14
O
15
P
16
Q
17
R
18
S
19
T
20
U
21
V
22
W
23
X
24
Y
25
Z
26


c:\users\shakt\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
IMG_SIZE = 96
LR = 1e-3  #.001 learing rate

nb_classes=28

MODEL_NAME = 'handsign.model'

def one_hot_targets_(labels_dense,nb_classes):
    targets = np.array(Y).reshape(-1)
    print(targets)
    one_hot_targets = np.eye(nb_classes)[targets]
    return one_hot_targets

In [9]:
train_data = np.load('train_data.npy',encoding="latin1",allow_pickle=True)
# test_data = np.load('test_data.npy',encoding="latin1")

train = train_data[:]
test = train_data[:100]

print('traindatlen:'+str(len(train)))
print('testdatalen:'+str(len(test)))

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
Y1=one_hot_targets_(Y,nb_classes)

traindatlen:13014
testdatalen:100
[ 0  5 17 ...  9  0 10]


In [10]:
IMG_SIZE = 96
LR = 1e-3

nb_classes=28

MODEL_NAME = 'handsign.model'

In [11]:
def cnn_model():
    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
    convnet = conv_2d(convnet, 8, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)
    convnet = conv_2d(convnet, 16, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 32, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)


    convnet = conv_2d(convnet, 64, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 128, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = fully_connected(convnet, 1024, activation='relu')
    convnet = dropout(convnet, 0.8)
    
    convnet = fully_connected(convnet, nb_classes, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')
    
    model = tflearn.DNN(convnet, tensorboard_dir='log')
    
    return model

In [12]:
IMG_SIZE = 96
LR = 1e-3  #.001 learing rate

nb_classes=28

MODEL_NAME = 'handsign.model'

In [13]:
def one_hot_targets_(labels_dense,nb_classes):
    targets = np.array(Y).reshape(-1)
    print(targets)
    one_hot_targets = np.eye(nb_classes)[targets]
    return one_hot_targets

In [14]:
train_data = np.load('train_data.npy',encoding="latin1",allow_pickle=True)
# test_data = np.load('test_data.npy',encoding="latin1")

train = train_data[:]
test = train_data[:100]

print('traindatlen:'+str(len(train)))
print('testdatalen:'+str(len(test)))

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
Y1=one_hot_targets_(Y,nb_classes)
# 
# print('max y'+str(max(Y)))
# print('min y'+str(min(Y)))
print('val y'+str(Y1))
print('len X:'+str(len(X)))
print('len Y:'+str(len(Y)))
test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
test_y1=one_hot_targets_(test_y,nb_classes)
test_y=test_y1
Y=Y1
print('test_x:'+str(len(test_x)))
print('test_y:'+str(len(test_y)))
print('val y'+str(test_y1))

traindatlen:13014
testdatalen:100
[ 0  5 17 ...  9  0 10]
val y[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
len X:13014
len Y:13014
[ 0  5 17 ...  9  0 10]
test_x:100
test_y:13014
val y[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
model=cnn_model()

model.fit({'input': X}, {'targets': Y}, n_epoch=15, validation_set=({'input': test_x}, {'targets': test_y}), 
snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)

Training Step: 3059  | total loss: 1.14144 | time: 22.784s
| Adam | epoch: 015 | loss: 1.14144 - acc: 0.8701 -- iter: 12992/13014
Training Step: 3060  | total loss: 1.06669 | time: 24.045s
| Adam | epoch: 015 | loss: 1.06669 - acc: 0.8721 | val_loss: 0.75527 - val_acc: 0.9000 -- iter: 13014/13014
--
INFO:tensorflow:C:\Users\shakt\handsign.model is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
score = model.evaluate(test_x, test_y)
print('Test accuarcy: %0.2f%%' % (score[0] * 100))
print(color_model)

Test accuarcy: 90.00%
1


In [17]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

INFO:tensorflow:Restoring parameters from C:\Users\shakt\handsign.model
model loaded!


In [ ]:
out_label=['U', 'T', 'E', 'R', 'BKSP', 'Q', 'D', 'BLNK1', 'G', 'SPC', 'I', 'F', 'O', 'C', 'W', 'Y', 'N', 'V', 'H', 'H', 'P', 'A', 'S', 'L', 'K', 'X', 'BLNK', 'B', ]
pre=[]

s=''
cchar=[0,0]
c1=''

# initialize weight for running average
aWeight = 0.5

# get the reference to the webcam
camera = cv2.VideoCapture(0)

# region of interest (ROI) coordinates
top, right, bottom, left = 170, 150, 425, 450

# initialize num of frames
num_frames = 0

flag=0
flag1=0
